# Introduction to Rebound Question 2/3

In [ ]:
import numpy as np
import rebound
import random

a_b, e_b = 1.,0.5
a_p, e_p = 4.,0.2
mu = 0.5


In [ ]:
def setupSimulation(ab,eb,ap,ep,mu):
    sim = rebound.Simulation() #sim = rebound.Simulation()
    
    a_b,e_b = ab,eb
    m1 = 1.
    
    sim.add(m=m1, hash = "Star1") 
    
    m2 = (m1*mu)/(1-mu)
    inc_b = random.random()*np.pi
    Omega_b = random.random()*2*np.pi
    f_b=np.random.rand()*2.*np.pi
    sim.add(m =m2, a= a_b, e=e_b, inc =inc_b,Omega=Omega_b,f=f_b, hash = "Star2") 
    
    a_p,e_p = ap,ep
    inc_p = random.random()*np.pi
    Omega_p = random.random()*2*np.pi
    f_p=np.random.rand()*2.*np.pi
    sim.add(m=0.,a=a_p,e=e_p,inc=inc_p,Omega=Omega_p, f=f_p, hash = "Planet1")
    
    sim.move_to_com()
    return sim


In [ ]:
sim = setupSimulation(a_b,e_b,a_p,e_p,mu)
sim.status()

In [ ]:
sim = setupSimulation(a_b,e_b,a_p,e_p,mu)
sim.exit_max_distance = 1000*a_b

Torb = 2.*np.pi
Noutputs = 1000
Norb = 1e4 
Tmin = Torb 
Tmax = 30*Torb

times = np.linspace(Tmin, Tmax, Noutputs)
xS1,yS1 = np.zeros(Noutputs), np.zeros(Noutputs)
xS2,yS2 = np.zeros(Noutputs), np.zeros(Noutputs)
xP1,yP1 = np.zeros(Noutputs), np.zeros(Noutputs)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
for i,time in enumerate(times):
    try:
        sim.integrate(time)  
    except rebound.Escape as error:
        print(error)
        for j in range(sim.N):
            p = sim.particles[j]
            d2 = p.x**2 + p.y**2 + p.z**2
            if d2>sim.exit_max_distance**2:
                sim.exit(1) # cache index rather than remove here since our loop would go beyond end of particles array
    xS1[i] = sim.particles["Star1"].x
    yS1[i] = sim.particles["Star1"].y
    xS2[i] = sim.particles["Star2"].x
    yS2[i] = sim.particles["Star2"].y
    xP1[i] = sim.particles["Planet1"].x
    yP1[i] = sim.particles["Planet1"].y

fig = plt.figure(figsize=(5,5))
ax = plt.subplot(111)
ax.set_xlim([-6,6])
ax.set_ylim([-4,4])


plt.plot(xS1, yS1,xS2,yS2,xP1,yP1)

#plot final orbit for each?

# for i in range(len(xP1)):
#     print(xP1[i])
    


In [ ]:
%matplotlib inline
fig, ax = rebound.OrbitPlot(sim)

# How does your plot compare to REBOUND??s OrbitPlot? Why might
# there be any differences? What happens if you set ap/ab = 3,
# and why? - planet exited the system?

# CLASSIC RESULTS

In [ ]:
import numpy as np
import rebound
import random
from multiprocessing import Pool 

In [ ]:
def Simulation(par):
    sim = rebound.Simulation() #sim = rebound.Simulation()
    e_b,a_p = par
    a_b = 1.
    m1 =1.
   
    sim.add(m=m1, hash = "Star1") 
    
    mu = 0.5
    m2 = (m1*mu)/(1-mu)
    inc_b = random.random()*np.pi
    Omega_b = random.random()*2*np.pi
    f_b=np.random.rand()*2.*np.pi
    sim.add(m =m2, a= a_b, e=e_b, inc =inc_b,Omega=Omega_b,f=f_b,  hash = "Star2")
    
    e_p = 0
    inc_p = random.random()*np.pi
    Omega_p = random.random()*2*np.pi
    f_p=np.random.rand()*2.*np.pi
    sim.add(m=0.,a=a_p,e=e_p,inc=inc_p,Omega=Omega_p, f=f_p, hash = "Planet1")
    
    #im.move_to_com()
    #sim.exit_max_distance = 1000*a_b
    max_dist = 100*a_b

    Torb = 2.*np.pi
    Noutputs = 100
    Norb_max = 1e4 
    Tmin = 0.
    Tmax = Norb_max*Torb
    times = np.linspace(Tmin, Tmax, Noutputs)
    tmax = np.zeros[3]
    for j in range(sim.N):
        p = sim.particles[j]
        for i,time in enumerate(times):
            sim.integrate(time)            
            d2 = p.x**2 + p.y**2 + p.z**2
            if(d2 > max_dist**2).any():
                break
    tmax[j] = time
        
                
     
    
    
#      for i,time in enumerate(times):
#         sim.integrate(time)
#         for j in range(sim.N):
#             p = sim.particles[j]
#             d2 = p.x**2 + p.y**2 + p.z**2
#             if(d2 > max_dist**2).any():
#                 break
#         tmax[j] = time
        
    
               
                
    return tmax



# sim = setupSimulation(a_b,e_b,a_p,e_p,mu)
# sim.status()



In [ ]:
random.seed(1)
N = 101
ab = 1
ebs = np.linspace(0.,0.7,N)
aps = ab*np.linspace(1.,5.,N)
params = [(eb,ap) for eb in ebs for ap in aps]

pool = rebound.InterruptiblePool()
stime = pool.map(Simulation,params) #survival times
stime = np.array(stime).reshape(N,N)
stime = np.nan_to_num(stime)



%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.colors import LogNorm
import matplotlib

t,ax = plt.subplots(1,1,figsize=(7,5))
extent=[ebs.min(), ebs.max(), aps.min(), aps.max()]

ax.set_xlim(extent[0], extent[1])
ax.set_ylim(extent[2], extent[3])
ax.set_xlabel("e_b")
ax.set_ylabel("a_p")

im = ax.imshow(stime, norm=LogNorm(vmax=1., vmin=1.e-16), aspect='auto', origin="lower", interpolation='nearest', cmap="RdYlGn_r", extent=extent)
cb = plt.colorbar(im, ax=ax)
cb.solids.set_rasterized(True)
cb.set_label("e_b vs. a_p")

    
